In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import linear_model

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# Generate consistent data for the dataset
rows = 500
pedestrian_walking = np.random.randint(0, 50, rows)
vehicles = np.random.randint(0, 40, rows)
weather_condition = np.random.choice(['Clear', 'Rain', 'Snow', 'Fog'], rows)
hour_of_day = np.random.randint(0, 24, rows)

# Adjust ambient light based on conditions and hour of the day
ambient_light = []
for i in range(rows):
    if pedestrian_walking[i] > 30 or vehicles[i] > 20:
        ambient_light.append(np.random.uniform(0.8, 1))  # High light requirement for high pedestrian or vehicle density
    elif weather_condition[i] == 'Fog':
        ambient_light.append(np.random.uniform(0.6, 0.8))  # Moderate light requirement for foggy weather
    elif weather_condition[i] == 'Rain':
        ambient_light.append(np.random.uniform(0.7, 0.9))  # Moderate to high light requirement for rainy weather
    elif weather_condition[i] == 'Snow':
        ambient_light.append(np.random.uniform(0.7, 0.9))  # Moderate to high light requirement for snowy weather
    elif 18 <= hour_of_day[i] or hour_of_day[i] < 5:
        ambient_light.append(np.random.uniform(0.8, 1))  # High light requirement during evening and night hours
    elif hour_of_day[i] >= 5 and hour_of_day[i] < 17:
        ambient_light.append(np.random.uniform(0, 0.4))  # Low light requirement during daytime hours
    else:
        ambient_light.append(np.random.uniform(0.4, 0.6))  # Moderate light requirement for other conditions

# Create DataFrame
data = {
    'no_of_pedestrian_walking': pedestrian_walking,
    'vehicles': vehicles,
    'weather_condition': weather_condition,
    'hour_of_day': hour_of_day,
    'ambient_light': ambient_light
}

df = pd.DataFrame(data)

# Display the dataset
print(df)


     no_of_pedestrian_walking  vehicles weather_condition  hour_of_day  \
0                          43        28              Rain           22   
1                          15        35              Snow           23   
2                           6        31             Clear           12   
3                           8        28             Clear            1   
4                          13        18              Snow            1   
..                        ...       ...               ...          ...   
495                        11        22              Snow           14   
496                        20        15              Snow           20   
497                        14        31              Rain           10   
498                        42         8              Rain           16   
499                        46         9               Fog           14   

     ambient_light  
0         0.823623  
1         0.911783  
2         0.914369  
3         0.809010  
4     

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score

# Split dataset into features (X) and target variables (y)
X = df[['no_of_pedestrian_walking', 'vehicles', 'weather_condition', 'hour_of_day']]
y_regressor = df['ambient_light']
y_classifier = (df['ambient_light'] > 0.5).astype(int)  # Binary classification for light requirement

# Convert categorical variables to dummy variables
X = pd.get_dummies(X)

# Split dataset into training and testing sets
X_train, X_test, y_train_regressor, y_test_regressor, y_train_classifier, y_test_classifier = train_test_split(X, y_regressor, y_classifier, test_size=0.2, random_state=42)

# Train regressor model
regressor_model = RandomForestRegressor(n_estimators=100, random_state=42)
regressor_model.fit(X_train, y_train_regressor)

# Train classifier model
classifier_model = RandomForestClassifier(n_estimators=100, random_state=42)
classifier_model.fit(X_train, y_train_classifier)

# Predictions
y_pred_regressor = regressor_model.predict(X_test)
y_pred_classifier = classifier_model.predict(X_test)

# Evaluate regressor model
mse = mean_squared_error(y_test_regressor, y_pred_regressor)
print("Regressor MSE:", mse)

# Evaluate classifier model
accuracy = accuracy_score(y_test_classifier, y_pred_classifier)
print("Classifier Accuracy:", accuracy*100)


Regressor MSE: 0.004595685440419311
Classifier Accuracy: 1.0


In [ ]:
def predict_light_requirement(classifier_model, regressor_model):
    # Ask user to input values
    pedestrian_walking = int(input("Enter the number of pedestrians walking: "))
    vehicles = int(input("Enter the number of vehicles: "))
    weather_condition = input("Enter the weather condition (Clear/Rain/Snow/Fog): ").capitalize()
    hour_of_day = int(input("Enter the hour of the day (0-23): "))

    # Convert weather condition to dummy variable
    weather_clear = 1 if weather_condition == 'Clear' else 0
    weather_rain = 1 if weather_condition == 'Rain' else 0
    weather_snow = 1 if weather_condition == 'Snow' else 0
    weather_fog = 1 if weather_condition == 'Fog' else 0

    # Create feature array
    feature = [[pedestrian_walking, vehicles, hour_of_day, weather_clear, weather_rain, weather_snow, weather_fog]]

    # Predict light requirement using classifier model
    light_requirement = "Yes" if classifier_model.predict(feature)[0] == 1 else "No"

    # Predict ambient light using regressor model
    ambient_light = regressor_model.predict(feature)[0]

    return light_requirement, ambient_light

# Example usage
light_requirement, ambient_light = predict_light_requirement(classifier_model, regressor_model)
print("Turn On Light?: ", light_requirement)
print("Predicted Ambient Light Level: {:.2f}%".format(ambient_light * 100))

Enter the number of pedestrians walking: 20
Enter the number of vehicles: 10
Enter the weather condition (Clear/Rain/Snow/Fog): clear
Enter the hour of the day (0-23): 18
Turn On Light?:  Yes
Predicted Ambient Light Level: 68.45%


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
